In [10]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import json
import random
device = "cuda" # the device to load the model onto

In [3]:

model = AutoModelForCausalLM.from_pretrained(
    "path2Qwen2",  # path2Qwen2
    torch_dtype="auto",
    device_map="auto"
)
tokenizer = AutoTokenizer.from_pretrained("path2Qwen2")


Loading checkpoint shards: 100%|██████████| 4/4 [00:06<00:00,  1.56s/it]
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [34]:
def chat(system, prompt, model=model, tokenizer=tokenizer):

    messages = [
    {"role": "system", "content": system},
    {"role": "user", "content": prompt}
        ]
    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )
    model_inputs = tokenizer([text], return_tensors="pt").to(device)

    generated_ids = model.generate(
        model_inputs.input_ids,
        max_new_tokens=512
    )
    generated_ids = [
        output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
    ]

    return tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

In [5]:
response = chat('hello')

In [36]:
with open('./Eval/dataset/GAOKAO_math.jsonl', 'r') as file:
    content = json.load(file)
content

[{'instruction': '请您根据以下问题，参考相关参考选项进行作答。',
  'input': '问题是:已知集合 A = {x | -5 < x^3 < 5}, B = {-3, -1, 0, 2, 3}，则 A ∩ B = (  ), 选项是:A: {-1, 0}, B: {2, 3}, C: {-3, -1, 0}, D: {-1, 0, 2}, 您的答案是:',
  'output': 'A'},
 {'instruction': '请您根据以下问题，参考相关参考选项进行作答。',
  'input': '问题是:若 z / (z - 1) = 1 + i，则 z = (  ), 选项是:A: -1 - i, B: -1 + i, C: 1 - i, D: 1 + i, 您的答案是:',
  'output': 'C'},
 {'instruction': '请您根据以下问题，参考相关参考选项进行作答。',
  'input': '问题是:已知向量 ā = (0, 1), b̄ = (2, x)，若 b̄ ⊥ (b̄ - 4ā)，则 x = (  ), 选项是:A: -2, B: -1, C: 1, D: 2, 您的答案是:',
  'output': 'D'},
 {'instruction': '请您根据以下问题，参考相关参考选项进行作答。',
  'input': '问题是:已知 cos(α + β) = m, tanαtan β = 2，则 cos(α - β) = (  ), 选项是:A: -3m, B: -m/3, C: m/3, D: 3m, 您的答案是:',
  'output': 'A'},
 {'instruction': '请您根据以下问题，参考相关参考选项进行作答。',
  'input': '问题是:已知圆柱和圆锥底面半径相等，侧面半径相等，且它们的高均为 √3，则圆锥的体积为 (  ), 选项是:A: 2√3π, B: 3√3π, C: 6√3π, D: 9√3π, 您的答案是:',
  'output': 'B'},
 {'instruction': '请您根据以下问题，参考相关参考选项进行作答。',
  'input': '问题是:已知函数 f(x) = { -x^2 - 2ax - a, x < 0; e^x +

In [37]:
idx = random.randint(0, len(content))
instruction, question, answer = content[idx].values()
prompt = '{question}'.format(question=question)
response = chat(instruction, prompt)
print("The question is:", prompt)
print("The prediction is: ", response)
print("The answer is: ", answer)

The question is: 问题是:已知 cos(α + β) = m, tanαtan β = 2，则 cos(α - β) = (  ), 选项是:A: -3m, B: -m/3, C: m/3, D: 3m, 您的答案是:
The prediction is:  为了找到 \(\cos(α - β)\)，我们可以利用三角恒等式和已知信息。已知：

1. \(\cos(α + β) = m\)
2. \(\tanα\tanβ = 2\)

首先，我们知道 \(\cos(α + β) = \cosα\cosβ - \sinα\sinβ = m\)。

其次，利用 \(\tanα\tanβ = 2\)，可以转换为 \(\frac{\sinα\sinβ}{\cosα\cosβ} = 2\)，从而得到 \(\sinα\sinβ = 2\cosα\cosβ\)。

现在，我们需要求 \(\cos(α - β) = \cosα\cosβ + \sinα\sinβ\)。

将 \(\sinα\sinβ\) 替换为 \(2\cosα\cosβ\)：

\[
\cos(α - β) = \cosα\cosβ + 2\cosα\cosβ = 3\cosα\cosβ
\]

我们还需要利用 \(\cos(α + β) = \cosα\cosβ - \sinα\sinβ = m\) 来表示 \(\cosα\cosβ\) 和 \(\sinα\sinβ\) 的关系。但是，我们注意到直接从给定的条件出发，我们可以通过另一种方式来简化问题，即利用和差化积公式：

\[
\cos(α - β) = \cosα\cosβ + \sinα\sinβ = (\cosα\cosβ - \sinα\sinβ) + 2\sinα\sinβ
\]

已知 \(\cos(α + β) = m\)，我们可以进一步使用和差化积公式，即：

\[
\cos(α + β) = \cosα\cosβ - \sinα\sinβ = m
\]

因此，我们有：

\[
\cos(α - β) = m + 2\sinα\sinβ
\]

由于 \(\sinα\sinβ = 2\cosα\cosβ\)，我们需要找到一个方法来表示 \(\sinα\sinβ\) 与 \(\cosα\cosβ\) 的关系来简化表达式。考虑到 \